#BASIC SQL COURSE

In the following sections, we will find the most basic and essential requirements for working with SQL. For example, the elementary statement to display the entire (non-distributed) database on the screen is simply.

In [0]:
%sql
SELECT * 
FROM platzi.alumnos

##1)-INDEXATION AND SELECTION

We create a query that SELECT all the elements of the database (*) FROM a table called "alumnos" from a platzi sql file. 

To select a bunch of specific elements we use FETCH which is equivalent to a LIMIT clause but in the ANSI norm of SQL, taking the first element ONLY starting at the FIRST row element of the table.

In [0]:
%sql

SELECT *
FROM platzi.alumnos
FETCH FIRST 1 ROWS ONLY;

The previous query is equivalent to.

In [0]:
%sql

SELECT*
FROM platzi.alumnos
LIMIT 1

The following query, uses de concept of "window functions", which are relationships between a group of data that we want to select and the entire table.

The next query uses a sub-selection from the whole elements of the table, using ROW_NUMBER() as a window function, that in this case, returns the register number independent ofany critiria that we impose. This function is applier OVER() the whole table and then assigns an allias AS row_id.

Finally, after declaring the source table, the selected group through the query is called "alumnos_with_row_num".

In [0]:
%sql

SELECT *
FROM (
	SELECT ROW_NUMBER() OVER() AS row_id, *
	FROM platzi.alumnos
) AS alumnos_with_row_num

The ROW_NUMBER() OVER() function assigns a unique sequential number to each row in the result set. In this case, it is aliased as "first_rows."
The * selects all columns from the "platzi.alumnos" table.

The inner query is treated as a derived table and aliased as "first_five" in the outer query:
    
SELECT *
FROM (inner_query) AS first_five

Finally, the outer query filters the results to only include rows where the "first_rows" value is less than or equal to 5:

WHERE first_rows <= 5

In [0]:
%sql

SELECT *
FROM platzi.alumnos
FETCH FIRST 5 ROWS ONLY

In [0]:
%sql
SELECT*
FROM platzi.alumnos
LIMIT 5

In [0]:
%sql

SELECT *
FROM(
  SELECT ROW_NUMBER() OVER() AS first_rows , *
  FROM platzi.alumnos
) AS first_five

WHERE first_rows <= 5

##2)-BASIC FILTERING AND SUB-QUERIES

We extract all the distinct registers within a table by using the command DISTINCT.

In [0]:
%sql

SELECT DISTINCT colegiatura
FROM platzy.alumnos AS a_1
WHERE 2 = (
  SELECT COUNT (DISTINCT colegitura)
  FROM platzi.alumnos AS a_2
  WHERE a_1.colegiatura <= a_2.colegiatura
)

In [0]:
%sql
SELECT DISTINCT colegiatura
FROM platzi.alumnos
ORDER BY colegiatura DESC
LIMIT 1 OFFSET 1

In [0]:
%sql
SELECT DISTINCT colegiatura
FROM platzi.alumnos
WHERE tutor_id = 20
ORDER BY colegiatura DESC
LIMIT 1 OFFSET 1

In the following query, we select all the rows (together with all their columns) of a table that meet the criterion of having in their category "colegiatura", the second highest "colegiatura" value if and only if their "tutor_id" value is equal to 20.

The inner query is treated as a derived table and aliased as "second_mayor_colegiatura" in the outer query:

INNER JOIN (inner_query) AS second_mayor_colegiatura
ON datos_alumnos.colegiatura = second_mayor_colegiatura.colegiatura

Then, the outer query retrieves all columns (*) from the "platzi.alumnos" table, which is aliased as "datos_alumnos.". The INNER JOIN clause is used to join the "platzi.alumnos" table with the derived table on the condition that the "colegiatura" values are equal.

In other words, The INNER JOIN operation combines the rows from the "platzi.alumnos" table (datos_alumnos) with the rows from the derived table (second_mayor_colegiatura) based on the matching values of the "colegiatura" column.

The ON clause specifies the condition for the join operation. In this case, it compares the "colegiatura" values of datos_alumnos and second_mayor_colegiatura to determine which rows should be included in the result set.

By joining the tables on the condition datos_alumnos.colegiatura = second_mayor_colegiatura.colegiatura, only the rows from datos_alumnos that have a matching "colegiatura" value in second_mayor_colegiatura will be included in the final result. Essentially, it filters datos_alumnos based on the "colegiatura" value to include only the rows that satisfy the join condition.

In [0]:
%sql

SELECT *
FROM platzi.alumnos AS datos_alumnos
INNER JOIN(
  SELECT DISTINCT colegiatura
  FROM platzi.alumnos
  WHERE tutor_id = 20
  ORDER BY colegiatura DESC
  LIMIT 1 OFFSET 1
) AS second_mayor_colegiatura
ON datos_alumnos.colegiatura = second_mayor_colegiatura.colegiatura

The same double filtering task can be accomplished in a simpler way by using WHERE, where you specify that the return values of a field from a table are equal to those that fulfill an internal query.

In [0]:
%sql

SELECT *
FROM platzi.alumnos AS datos_alumnos
WHERE colegiatura = (
	SELECT DISTINCT colegiatura
	FROM platzi.alumnos
	WHERE tutor_id = 20
	ORDER BY colegiatura DESC
	LIMIT 1 OFFSET 1
)

To return the second half of the table we can do the following:

In [0]:
%sql
SELECT *
FROM platzi.alumnos
OFFSET(
  SELECT COUNT(*)/2
  FROM platzi.alumnos
)

##3)-SELECTION FROM A SUBSET OF OPTIONS

When the queries are made on a very large data and the query is very complex, it is advisable to add a row_id to each element of the table that has an indicator given sequentially by ourselves, this way it will be easier to filter by artificial id than by some criteria that is very difficult to trace. An example of indexing by user and filtering by artificial id is the following.

In [0]:
%sql

SELECT *
FROM(
  SELECT ROW_NUMBER() OVER() AS row_id, *
  FROM platzi.alumnos
)AS alumnos_with_row_num
WHERE row_id IN (1,5,10,12,15,20)

In [0]:
%sql

SELECT * 
FROM platzi.alumnos
WHERE id IN (
  SELECT id
  FROM platzi.alumnos
  WHERE tutor_id = 30
)

In [0]:
%sql

SELECT * 
FROM platzi.alumnos
WHERE id IN (
  SELECT id
  FROM platzi.alumnos
  WHERE tutor_id = 30
    AND carrera_id = 31
)

In [0]:
%sql

SELECT *
FROM platzi.alumnos
WHERE id IN(22)

To negate the inner query and obtain all the entries excepting the ones that satifies the inner query, we simply add NOT to the outer query.

In [0]:
%sql

SELECT * 
FROM platzi.alumnos
WHERE id NOT IN (
  SELECT id
  FROM platzi.alumnos
  WHERE tutor_id = 30
)

##4)-EXTRACT AND TIMESTAMPS

To extract specific information we use EXTRACT, for example, if we want to obtain only a table with the incorporation year we use the following query.

In [0]:
%sql

SELECT EXTRACT(YEAR FROM fecha_incorporación) ASY y_in
FROM platzi.alumnos 

We can achieve the same result but this time, using an specific function that extracts date times called DATE_PART, where we need to stipulate the field to extact and the column of the table containing the date to query.

In [0]:
%sql

SELECT DATE_PART('YEAR', fecha_incorporacion) AS anio_incorporacion
From platzi.alumnos

The following query, is useful to for example, reorganize the date tieme and structure it in a column per date tiem item.

In [0]:
%sql

SELECT DATE_PART('YEAR', fecha_incorporacion) AS mes_incorporacion,
       DATE_PART('MONTH', fecha_incorporacion) AS mes_incorporacion,
       DATE_PART('DAY', fecha_incorporacion) AS dia_incorporacion
From platzi.alumnos

And the corresponding day time time stamp is queried as.

In [0]:
%sql

SELECT DATE_PART('HOUR', fecha_incorporacion) AS hour_incorporacion,
       DATE_PART('MINUTE', fecha_incorporacion) AS minute_incorporacion,
       DATE_PART('SECOND', fecha_incorporacion) AS second_incorporacion
From platzi.alumnos

Applying these queries as filters, we can use EXTRACT() or DATE_PART() as the proyection conditon.

In [0]:
%sql

SELECT *
FROM platzi.alumnos
WHERE (EXTRACT(YEAR FROM fecha_incorporacion)) = 2019

In [0]:
%sql

SELECT *
FROM platzi.alumnos
WHERE (DATE_PART('YEAR',fecha_incorporacion)) = 2019

In [0]:
%sql

SELECT *
FROM(
  SELECT *,
	DATE_PART('YEAR',fecha_incorporacion) AS ano_incorporacion
  FROM platzi.alumnos
      
) AS alumnos_con_año

Once we have applied a subquery to select the data with a extra year column, we can filter it according to this field.

In [0]:
%sql

SELECT *
FROM(
  SELECT *,
	DATE_PART('YEAR',fecha_incorporacion) AS ano_incorporacion
  FROM platzi.alumnos
      
) AS alumnos_con_año
WHERE ano_incorporacion = 2020

we can make a filtering process according to two conditions defined in the inner query by using the operator AND in the following way.

In [0]:
%sql

SELECT *
FROM(
  SELECT *,
	DATE_PART('YEAR',fecha_incorporacion) AS ano_incorporacion,
	DATE_PART('MONTH',fecha_incorporacion) AS mes_incorporacion
  FROM platzi.alumnos
      
) AS alumnos_con_año
WHERE (
	ano_incorporacion = 2018
	AND 
	mes_incorporacion = 5
)

##5)-FINDING DUPLICATES

To find duplicates, we need to create two allias or instances of the same table in order to make comparison of the table with itself. Then, we stipulate the column to check and then we check for equalities. The simpliest case and a good first practice, is to check when the id is repeated.

In [0]:
%sql

SELECT platzy.alumnos AS ou
WHERE(
  SELECT COUNT(*)
  FROM platzy.alumnos AS inr
  WHERE ou.id = inr.id
) >1

To cast (transform) the entire table into a comma separated text, where each row is colapsated into a comma separated tuple of elements, we execute the following.

In [0]:
%sql

SELECT (platzi.alumnos.*)::text
FROM platzi.alumnos
GROUP BY platzi.alumnos.*

Making this first conversion, we can implement HAVING COUNT(*) in order to return those tuples that has a count number bigger than one. Nevertheless, due to the id presence, we need to select all themain object characteristics in order to evaluate duplication according to those fields.

In [0]:
%sql

SELECT (
  platzi.alumnos.nombre,
  platzi.alumnos.apellido,
  platzi.alumnos.email,
  platzi.alumnos.colegiatura,
  platzi.alumnos.fecha_incorporacion,
  platzi.alumnos.carrera_id,
  platzi.alumnos.tutor_id
  )::text, COUNT(*)
FROM platzi.alumnos
GROUP BY (
  platzi.alumnos.nombre,
  platzi.alumnos.apellido,
  platzi.alumnos.email,
  platzi.alumnos.colegiatura,
  platzi.alumnos.fecha_incorporacion,
  platzi.alumnos.carrera_id,
  platzi.alumnos.tutor_id
  )
HAVING COUNT(*) >1

We can achieve the same result in a more confortable way by using subqueries and windoe functions.

In [0]:
%sql

SELECT *
FROM(
	SELECT id,
	ROW_NUMBER() OVER(
		PARTITION BY(
			nombre,
			apellido,
			email,
			colegiatura,
			fecha_incorporacion,
			carrera_id,
			tutor_id
		)
		ORDER BY id ASC		
	) AS row,*
	FROM platzi.alumnos
) AS duplicados

WHERE duplicados.row > 1


To see an example of how this partitionated query works, imagine the following table with one suplicate.

| id | nombre  | apellido |     email      | colegiatura | fecha_incorporacion   | carrera_id | tutor_id |
|---|---|---|---|---|---|---|---|
|  1 | John    | Doe      | john@example.com|    2000     | 2023-01-01 00:00:00   |     1      |    10    |
|  2 | Jane    | Smith    | jane@example.com|    1500     | 2023-02-01 00:00:00   |     2      |    20    |
|  3 | John    | Doe      | john@example.com|    2000     | 2023-03-01 00:00:00   |     1      |    10    |
|  4 | Mike    | Brown    | mike@example.com|    1800     | 2023-04-01 00:00:00   |     3      |    10    |
|  5 | Emma    | Davis    | emma@example.com|    1700     | 2023-05-01 00:00:00   |     2      |    30    |
|  6 | Jane    | Smith    | jane@example.com|    1500     | 2023-06-01 00:00:00   |     2      |    20    |

Then, the PARTITION BY  clause would generate a group by each entry class, such that duplicated objects.


| id | row | nombre  | apellido |     email      | colegiatura | fecha_incorporacion   | carrera_id | tutor_id |
|---|---|---|---|---|---|---|---|---|
|  1 |  1  | John    | Doe      | john@example.com|    2000     | 2023-01-01 00:00:00   |     1      |    10    |
|  2 |  1  | John    | Doe      | john@example.com|    2000     | 2023-01-01 00:00:00   |     1      |    10    |

Note that ROW_NUMER() is added as a new column such that the values in the "row" column should be sequential numbers within each partition. In the example you mentioned, it should be 1 and 2 for the rows that are considered duplicates.

In order to delete the duplicates, we need to extract only the id of the duplicate, to this end, we modify the previous sentence and then use it as a subquery in a WHERE filter.

In [0]:
%sql

DELETE FROM platzi.alumnos
WHERE id IN (

	SELECT id
    FROM(
	    SELECT id,
	    ROW_NUMBER() OVER(
		    PARTITION BY( 
			    nombre,
			    apellido,
			    email,
			    colegiatura,
			    fecha_incorporacion,
			    carrera_id,
			    tutor_id
		    )
		    ORDER BY id ASC		
	    ) AS row
	    FROM platzi.alumnos
    ) AS duplicados

  WHERE duplicados.row > 1
)

##6)-RANGE SELECTORS

Range selectors are filters that extracts all the registers finded within a range of values. The simpliest implementation is.

In [0]:
%sql

SELECT *
FROM platzi.alumnos
WHERE tutor_id IN (1,2,3,4)

Also, we can return the values between a certain range in two maners.

In [0]:
%sql

SELECT *
FROM platzi.alumnos
WHERE (
  tutor_id >= 1
  AND
  tutor_id <=10
)

In [0]:
%sql

SELECT *
FROM platzi.alumnos
WHERE tutor_id BETWEEN 1 AND 10

Nevertheless, range selection not only serves to filter the data directly, it is also usefull as a conditioner. For example, to check if a condition holds.

In [0]:
%sql
SELECT int4range(10,20) @>3

Or to check if two ranges intersect to each other.

In [0]:
%sql
SELECT numrange(11.2,22.2) && numrange(20.0,30.0)

Or to select the largest value of a range.

In [0]:
%sql
SELECT UPPER(int4range(15,26))

Or to know the upper and lower intersection limits of a range of values.

In [0]:
%sql
SELECT int4range(1,18) * int4range(14,40)

Or to know if a range is empty.

In [0]:
%sql
SELECT ISEMPTY( numrange(1,8) )

Lets see some interesting examples of the uses cases of range selectors to filter the data in a more personalized way.

In [0]:
%sql

SELECT * 
FROM platzi.alumnos
WHERE int4range(10,20) @> tutor_id

In [0]:
%sql

SELECT numrange(
  (SELECT MIN(tutor_id) FROM platzi.alumnos),
  (SELECT MAX(tutor_id) FROM platzi.alumnos)
) * numrange(
  (SELECT MIN(carrera_id) FROM platzi.alumnos),
  (SELECT MAX(carrera_id) FROM platzi.alumnos)
)

In this sense, lets see how to extract the maximum and minimun from a table.

In [0]:
%sql

SELECT fecha_incorporacion
FROM platzi.alumnos
ORDER BY fecha_incorporacion DESC
LIMIT 1

In the next query, we will use one of the columns to organize the records by date of incorporation. Wanting to organize the table by race incorporation date, we select race_id as the organized return criteria, then, we use MAX to find in the date field by race, then, we group the records according to the most recent or maximum record date found by MAX for each race id.

In [0]:
%sql

SELECT carrera_id, MAX(fecha_incorporacion)
FROM platzi.alumnos
GROUP BY carrera_id
ORDER BY carrera_id

Another example, lets return the first alphabetically placed name registered for a tutor.

In [0]:
%sql

SELECT tutor_id, MIN(nombre)
FROM platzi.alumnos
GROUP BY tutor_id
ORDER BY tutor_id

##7)-SELF JOINTS

A self-join in SQL is a type of join operation where a table is joined with itself. It involves combining rows from the same table based on a related column or condition. In other words, a self-join treats a single table as if it were two separate tables, creating aliases or table aliases to differentiate between them.

Self-joins can be useful when you have a table that contains hierarchical or recursive data, and you need to compare or match rows within the same table.

To perform a self-join, you need to use table aliases to create separate references to the same table within the query. By using different aliases, you can treat the table as if it were two separate entities, allowing you to join them based on the desired condition.

For example, imagine that there is recursion in the data (which is not the same as duplication) if, for example, some of the students in a Platzi program were not only receiving education but also teaching some of the courses in other programs.

In [0]:
%sql

SELECT a.nombre,
       a.apellido,
       t.nombre,
       t.apellido
FROM platzi.alumnos AS a
  INNER JOIN platzi.alumnos AS t ON a.tutor_id = t.id

For ease of visualization and exloration, we can use CONCAT() to concatenate the name and the last name of both, students and tutors.

In [0]:
%sql

SELECT CONCAT(a.nombre,' ',a.apellido) AS alumno,
       CONCAT(t.nombre,' ',t.apellido) AS tutor

FROM platzi.alumnos AS a
  INNER JOIN platzi.alumnos AS t ON a.tutor_id = t.id

Now, lets see the amount of students per tutor.

In [0]:
%sql

SELECT CONCAT(t.nombre,' ',t.apellido) AS tutor,
       COUNT(*) AS alumnos_por_tutor
       

FROM platzi.alumnos AS a
  INNER JOIN platzi.alumnos AS t ON a.tutor_id = t.id
 GROUP BY tutor
 ORDER BY alumnos_por_tutor DESC

The following query extracts the average of students per tutor.

In [0]:
%sql

SELECT AVG(alumnos_por_tutor) AS promedio_alumnos_por_tutor
From(
  SELECT CONCAT(t.nombre,' ',t.apellido) AS tutor,
       COUNT(*) AS alumnos_por_tutor
  FROM platzi.alumnos AS a
  INNER JOIN platzi.alumnos AS t ON a.tutor_id = t.id
  GROUP BY tutor
) AS alumnos_tutor 

##8)-SOLVING DIFFERENCES

Resolving differences with SQL refers to making right or left exclusions of two tables to be related according to a chosen field. As an example, let's do a left exclude to table a and c being the table to keep and where we will use c to exclude the information that both tables share.

First, we group the data according to a field for the left (principal) join to see how the data is before the relational exclusion.

In [0]:
%sql
---We group the data according to a field---

SELECT carrera_id, COUNT(*) AS cuenta
FROM platzi.alumnos
GROUP BY carrera_id
ORDER BY cuenta DESC

After, we delete some cells in the selected field in order to make a left joint exclusion according to the null values of the selected field.
Then, we make a LEFT JOIN such that we return the information of the two related tables such that right and union exclusion is applied, then, the outputed table is conformed only by the left information that is not covered by the right table.

In [0]:
%sql
---We delete a field---

DELETE FROM platzi.carreras
WHERE id BETWEEN 30 AND 40

---We exlude information with left join---

SELECT a.nombre,
       a.apellido,
	   a.carrera_id,
	   c.id,
	   c.carrera
FROM platzi.alumnos AS a
     LEFT JOIN platzi.carreras AS c
	 ON a.carrera_id = C.id
WHERE c.id IS NULL
ORDER BY a.carrera_id

##9)-TOTAL JOINS AND EX/INCLUSSIVE JOINS

<img src = "https://static.platzi.com/media/user_upload/3bs7C-8809ee87-52f8-45b8-9df2-0620145e1598.jpg" alt = "Encabezado" width = "80%">  </img>

In contrast to exclusive joins, a FULL OUTER JOIN joins two tables according to a select field. As seen in the following query, when a full join of tables is executed, the command will return all the data from both tables which means that the unique data from the principal and foreign table together with the common data from both tables will be presented in a single table.

This will imply that if there are missing fields in one of the tables, SQL will automatically wrap the resulting table so that the field to which the relationship is made is concise while the missing fields in the foreign table are filled with a null value.

In [0]:
%sql
---We fully join information from two tables---

SELECT a.nombre,
       a.apellido,
	   a.carrera_id,
	   c.id,
	   c.carrera
FROM platzi.alumnos AS a
     FULL OUTER JOIN platzi.carreras AS c
	 ON a.carrera_id = C.id
ORDER BY a.carrera_id

As we saw previously, an exclusive JOIN compares the information of two tables according to a specific field and, through an exclusion criterion, returns only the information not shared by left (LEFT JOIN) or by right (RIGHT JOIN).

On the other hand, if we want a non-total but inclusive JOIN, it is enough to eliminate the selection filter of the LEFT/RIGHT JOIN so that SQL returns the information of the main table together with the intersection between the main table and the foreign table.

If there are rows or entries in the foreign table that, according to the relational criteria, do not exist within the main table, SQL will return all those values from the main table that exist or do not exist within the foreign table, together with all those values that from the foreign table matches the main table.

This is the main difference between the LEFT/RIGHT JOIN and the FULL OUTER JOIN, since the latter simply returns all the information from the main and foreign table while the LEFT/RIGHT are in charge of executing certain "filtering" of the foreign table in case of having an inclusive JOIN.

In [0]:
%sql
---We delete a field---

DELETE FROM platzi.carreras
WHERE id BETWEEN 30 AND 40

---We include information with left join---

SELECT a.nombre,
       a.apellido,
	   a.carrera_id,
	   c.id,
	   c.carrera
FROM platzi.alumnos AS a
     LEFT JOIN platzi.carreras AS c
	 ON a.carrera_id = C.id

ORDER BY a.carrera_id

Let's see that in the following case, when executing a RIGHT JOIN, we will return all the information from the foreign table in such a way that if we also use fields from the main table as return criteria, their empty entries (not present in the right table) will be filled with null fields. On the other hand, the query will return a table that contains the information of the right table together with all the information of the left table that is contained or common with the right, being an inclusive JOIN

To make an exclusive JOIN, it is enough to add a WHERE statement that includes an exclusion criteria, for example, that returns all the fields of the right table such that in the left table, the relational field is empty.

In [0]:
%sql
---Inclusive right join---
SELECT a.nombre,
       a.apellido,
	   a.carrera_id,
	   c.id,
	   c.carrera
FROM platzi.alumnos AS a
    RIGHT JOIN platzi.carreras AS c
	ON a.carrera_id = c.id
ORDER BY c.id DESC

In [0]:
%sql
---Exclusive right join---
SELECT a.nombre,
       a.apellido,
	   a.carrera_id,
	   c.id,
	   c.carrera
FROM platzi.alumnos AS a
    RIGHT JOIN platzi.carreras AS c
	ON a.carrera_id = c.id

WHERE a.id IS NULL
ORDER BY c.id DESC

An INNER JOIN between two tables turns out to be the intersection of information between both tables involved, that is, INNER JOIN or simply JOIN, returns the fields that are common to both the left table and the right table.

In [0]:
%sql
---Intersection join---
SELECT a.nombre,
       a.apellido,
	     a.carrera_id,
	     c.id,
	     c.carrera
FROM platzi.alumnos AS a
     INNER JOIN platzi.carreras AS c
	   ON a.carrera_id = c.id
ORDER BY c.id DESC

Finally, to make a symmetric difference, we need to do a full JOIN, that is, we have to apply a FULL OUTER JOIN that contains a WHERE filter statement that excludes by means of a selection criteria all the information contained in the intersection between tables.

In [0]:
%sql
---Symmetric difference join---
SELECT a.nombre,
       a.apellido,
       a.carrera_id,
       c.id,
       c.carrera

FROM platzi.alumnos AS a
     FULL OUTER JOIN platzi.carreras AS c
     ON a.carrera_id = c.id

WHERE (a.id IS NULL
       OR
       c.id IS NULL
)

ORDER BY a.carrera_id DESC, c.id DESC

##10)-TRIANGULARIZATION

In SQL, triangularization refers to a technique used to transform a dataset into a triangular matrix or triangular form. The resulting triangular matrix has non-zero values only in the upper or lower triangular portion, while the other portion contains zeros.

Triangularization can be useful in various scenarios, particulary for solving certain types of problems or optimizing data storage and processing. Some applications and benefits of triangularization include:

1. Space optimization: triangular matrices can store data more efficently compared to general matrices since they only requere storage for non-zero values. 
2. Algorithm optimization: triangular matrices can simplify expedite certain mathematical operation an algorithm, for example, triangular matrices are commonly used in linear algebra, optimization techniques, and numerical methods to solve systems of equations among others.
3. Data analysis and visualization: Triangularization can aid in visualizing and analyzing patterns, dependencies, or relationships in the data. By representing the data in a triangular form, it can be easier to identify trends or spot irregularities, especially when dealing with hierarchical or hierarchical-like structures.
4. Performance optimization: Triangular matrices can lead to performance improvements in certain operations, such as matrix multiplications or solving linear systems, by reducing the number of computations required.

The simpliest form of triangularization in SQL is when we want to output data with progressively increasing lenght within each row.

In [0]:
%sql
---left padding string---
SELECT lpad('*', CAST(row_number AS int), '*')

---subquery ordering---
FROM(
  SELECT ROW_NUMBER() OVER(ORDER BY carrera_id) AS row_id,*
  FROM platzi.alumnos
) AS alumnos_with_row_num
WHERE row_id <= 5
ORDER BY carrera_id

The given SQL query generates a string of asterisks (*) based on the value of the row_id column, which is derived using the ROW_NUMBER() function. The query selects the first five rows from the "platzi.alumnos" table, ordered by the carrera_id column.

Here's a breakdown of the query:

1. SELECT lpad('*',CAST(row_id AS int),'*'): This selects a string of asterisks (*) generated using the lpad() function. The lpad() function is used to left-pad a string with a specified character (in this case, asterisks) to a specified length. The length is determined by casting the row_id column as an integer (CAST(row_id AS int)). So, for each row, the number of asterisks in the resulting string corresponds to the value of row_id.

2. FROM (SELECT ROW_NUMBER() OVER(ORDER BY carrera_id) AS row_id, * FROM platzi.alumnos) AS alumnos_with_row_num: This constructs a derived table or subquery named "alumnos_with_row_num". It selects all columns from the "platzi.alumnos" table and assigns a sequential number to each row using the ROW_NUMBER() function. The numbering is determined by the ORDER BY carrera_id clause, which orders the rows based on the carrera_id column.

3. WHERE row_id <= 5: This filters the results by selecting only the rows where the row_id is less than or equal to 5. This ensures that only the first five rows, based on the order specified in the ORDER BY clause, are included in the final result set.

4. ORDER BY carrera_id: This orders the result set by the carrera_id column in ascending order. This is the outer ORDER BY clause, which sorts the final result set of the query.

##11)-RANK GENERATION

In SQL we can generate rank of number very easily with generate_series(start,end,step).

In [0]:
%sql
SELECT *
FROM generate_series(1,10,2)

We can use generate_series() to either query or modify data. For example, using current_date we can use the current date of the year and then modify it by say summing values to it. In the following query, SQL automatically assumes that the quentity to add to the current date are DAYS.

In [0]:
%sql
SELECT current_date + s.a AS dates
---Create table s with column a from gs---
FROM generate_series(0,14,7) AS s(a)


Also, we can modify another parts of time series, like hours in a date stamp.

In [0]:
%sql
SELECT *
FROM generate_series('2023-06-26 00:00:00'::timestamp,
'2023-06-30 00:00:00','10 hours')

Or to query data through INNER JOIN.

In [0]:
%sql
--- GS Inner join ---
SELECT a.id,
       a.nombre,
	   a.apellido,
	   a.carrera_id,
	   s.a

FROM platzi.alumnos AS a
     INNER JOIN generate_series(0,10) AS s(a)
	 ON s.a = a.carrera_id
ORDER BY a.carrera_id

Another use, is to create ordinality

In [0]:
%sql
SELECT *
FROM generate_series(10,20,-2) WITH ordinality

In [0]:
%sql
SELECT lpad('-',CAST(ordinality AS ind),'*')
FROM generate_series(10,20,-2) WITH ordinality

##12)-REGULAR EXPRESIONS

Regular expressions, often referred to as regex, are powerful pattern-matching tools used to search, match, and manipulate text based on specified patterns. In SQL, regular expressions are supported by various database systems, and they provide additional flexibility and functionality for data manipulation and querying.

Here are some utilities and use cases of regular expressions in SQL:

1. Pattern matching: Regular expressions allow you to search for patterns within text data. For example, you can use regular expressions to find all rows in a table where a particular column value matches a specific pattern, such as email addresses, phone numbers, or certain word combinations.

2. Data validation: Regular expressions can be used to enforce data validation rules within SQL queries. You can validate inputs against predefined patterns to ensure they meet specific criteria, such as a valid format for dates, social security numbers, or ZIP codes.

3. Data extraction and transformation: Regular expressions enable you to extract specific parts of text or transform text data based on patterns. For example, you can extract substrings from a column value that match a particular pattern, or you can replace or modify parts of a string based on a regular expression pattern.

4. Complex string manipulation: Regular expressions provide a flexible and concise way to perform complex string manipulations. They allow you to perform operations such as string concatenation, splitting, formatting, and substitution based on patterns, making it easier to work with text data in SQL.

5. Query filtering and sorting: Regular expressions can be used in SQL queries to filter and sort data based on patterns. For instance, you can retrieve rows that match specific patterns or exclude rows that do not match certain criteria using regular expressions in the WHERE clause of a query.

In [0]:
 %sql
 SELECT email
 FROM platzi.alumnos
 WHERE email ~*'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [0]:
%sql
 SELECT email
 FROM platzi.alumnos
 WHERE email ~*'[A-Z0-9._%+-]+@google[A-Z0-9.-]+\.[A-Z]{2,4}'

##13)-WINDOW FUNCTIONS

Window functions in SQL correspond to column relationships with respect to a selection window made up of other rows, that is, they are functions that extract data according to the comparison of one row with the others based on a criterion.

In general, the window function is defined as an action (rank, average, sum, etc) being applied over a query using OVER() where this clause represents the selection window.

Average as a window function applied over  a column partitions based on the value classes of another column.

In [0]:
%sql
---Average of a partition of a column---
SELECT *,
       AVG(colegiatura) OVER(PARTITION BY carrera_id)
FROM platzi.alumnos

Average of the whole st of values of a column.

In [0]:
%sql
SELECT*,
       AVG(colegiatura) OVER ()
FROM platzi.alumnos

A sum taking over an ODER BY clause, this is a sum carried out not on a partition but based on an ordering stipulated by means of another attribute (external) apart from the one on which the window function is applied (internal), SQL will return the sum value as a value composed of the first sum carried out on the field of interest (internal) between the rows that have the same external field, plus the sum of all the rows above the current row. That is, it executes a cumulative addition from top to bottom, returning the same value to each row that has the same external criteria.

The following query, uses as the inner criteria the "colegiatura" row and as the external criteria the same "colegiatura" row.

In [0]:
%sql
SELECT*,
       SUM(colegiatura) OVER (ORDER BY colegiatura)
FROM platzi.alumnos

To more easily see the cumulative addition process, we execute a partition using "career_id" as external criteria and "tuition" as internal criteria.

In [0]:
%sql
SELECT*,
       SUM(colegiatura) OVER (PARTITION BY carrera_id ORDER BY colegiatura)
FROM platzi.alumnos

In the case of RANK, used as a window function in conjunction with an ORDER BY clause, partitions the data according to an external criteria (in the case of the following example) ranks the internal criteria in ascending or descending manner. However, there is a particularity, and that is that if there are multiple rows with the same rank, the next rank of the next class of internal column value will have the rank that it would have according to the number of times that a previous rank is repeated. That is, if in the case of ordering the "colegiatura" field by rank, there are 4 rows with the same "colegiatura" value such that all they would have a rank equal to 1, the rank for the next "colegiatura" value will be 5.

In [0]:
%sql
SELECT*,
       RANK() OVER (PARTITION BY carrera_id ORDER BY colegiatura DESC)
FROM platzi.alumnos

Nevertheless, the use of window functions must take into account the order in which the SQL statements are executed since the way SQL is written is not always the way it is executed, in essence, there is a predetermined order for it. For example, if we apply a WHERE clause to filter the range values as shown in the following example, the "brand_rank" table will not be recognized by SQL.

In [0]:
%sql
SELECT*,
       RANK() OVER (PARTITION BY carrera_id ORDER BY colegiatura DESC) AS brand_rank
FROM platzi.alumnos
WHERE brand_rank <3
ORDER BY carrera_id, brand_rank

To solve this, we need to use the windoe function within a subquery such that the outher query executes the filter clause WHERE.

In [0]:
%sql
SELECT*
FROM(
    SELECT *,
    RANK() OVER(PARTITION BY carrera_id ORDER BY colegiatura DESC) AS brand_rank
    FROM platzi.alumnos
) AS ranked_colegiaturas_by_carrer
WHERE brand_rank <3
ORDER BY brand_rank

##14)-PARTITIONS AND AGREGATIONS

To generate an id/field that is made up of the first, last, or nth value of a partitioned field, we respectively execute a query structure similar to the following examples.

In [0]:
%sql
---Give a sequential row id over a partition---
SELECT ROW_NUMBER() OVER(ORDER BY fecha_incorporacion) AS row_id,*
FROM platzi.alumnos

In [0]:
%sql
---Give a row id according to the first value---
SELECT FIRST_VALUE(colegiatura) OVER() AS row_id,*
FROM platzi.alumnos

In [0]:
%sql
---Give a row id according to the last value---
SELECT LAST_VALUE(colegiatura) OVER() AS row_id,*
FROM platzi.alumnos

In [0]:
%sql
---Give a row id according to the N-th value---
SELECT NTH_VALUE(colegiatura, 3) OVER(PARTITION BY carrera_id) AS row_id,*
FROM platzi.alumnos

As before,another partition can be carried aout by rank. The following query executes a RANK clause which performes a ranking of the rows with gaps.

In [0]:
%sql
---Give a sequential rank with gaps over a partition---
SELECT *,
        RANK() OVER(PARTITION BY carrera_id ORDER BY colegiatura DESC) AS colegiatura_rank
FROM platzi.alumnos
ORDER BY carrera_id, colegiatura_rank

In order to have a more natural ranking of the rows, we use the DENSE_RANK clause.

In [0]:
%sql
---Give a sequential rank without gaps over a partition---
SELECT *,
        DENSE_RANK() OVER(PARTITION BY carrera_id ORDER BY colegiatura DESC) AS colegiatura_rank
FROM platzi.alumnos
ORDER BY carrera_id, colegiatura_rank

Finally, PERCENT_RANK performes a RANK-based clause but tanking into account the percentages of the actual rank compared with the database size. Generally speaking it uses the formula:

(RANK - 1)/(total_rows - 1)

In [0]:
%sql
---Give a percentual rank without gaps over a partition---
SELECT *,
        PERCENT_RANK() OVER(PARTITION BY carrera_id ORDER BY colegiatura DESC) AS colegiatura_rank
FROM platzi.alumnos
ORDER BY carrera_id, colegiatura_rank